In [48]:
# Data cleaning and EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Preprocessing
from datetime import datetime 
from sklearn.preprocessing import StandardScaler

# Modelling
#import sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

### Extracting the Dataset

In [104]:
# Importing training data
tdata = pd.read_csv('Hotel-A-train.csv')
vdata = pd.read_csv('Hotel-A-validation.csv')
testdata = pd.read_csv('Hotel-A-test.csv')
tdata.shape , vdata.shape, testdata.shape
#dimensions match

((27499, 24), (2749, 24), (4318, 23))

In [9]:
tdata.columns , vdata.columns, testdata.columns
# columns match

(Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
        'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
        'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
        'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
        'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
        'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate'],
       dtype='object'),
 Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
        'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
        'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
        'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
        'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
        'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate'],
       dtype='object'),
 Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
 

In [10]:
tdata.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242


###  Check for Missing values

In [33]:
tdata.info() , vdata.info(), testdata.info()
tdata.isna().sum() , vdata.isna().sum(), testdata.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27499 entries, 0 to 27498
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Reservation-id          27499 non-null  int64 
 1   Gender                  27499 non-null  object
 2   Age                     27499 non-null  int64 
 3   Ethnicity               27499 non-null  object
 4   Educational_Level       27499 non-null  object
 5   Income                  27499 non-null  object
 6   Country_region          27499 non-null  object
 7   Hotel_Type              27499 non-null  object
 8   Expected_checkin        27499 non-null  object
 9   Expected_checkout       27499 non-null  object
 10  Booking_date            27499 non-null  object
 11  Adults                  27499 non-null  int64 
 12  Children                27499 non-null  int64 
 13  Babies                  27499 non-null  int64 
 14  Meal_Type               27499 non-null  object
 15  Vi

(Reservation-id            0
 Gender                    0
 Age                       0
 Ethnicity                 0
 Educational_Level         0
 Income                    0
 Country_region            0
 Hotel_Type                0
 Expected_checkin          0
 Expected_checkout         0
 Booking_date              0
 Adults                    0
 Children                  0
 Babies                    0
 Meal_Type                 0
 Visted_Previously         0
 Previous_Cancellations    0
 Deposit_type              0
 Booking_channel           0
 Required_Car_Parking      0
 Reservation_Status        0
 Use_Promotion             0
 Discount_Rate             0
 Room_Rate                 0
 dtype: int64,
 Reservation-id            0
 Gender                    0
 Age                       0
 Ethnicity                 0
 Educational_Level         0
 Income                    0
 Country_region            0
 Hotel_Type                0
 Expected_checkin          0
 Expected_checkout         0

### Feature Engineering
1. Expected_stay_days = Expected_checkout - Expected_checkin
2. Reservation_gap = Expected_checkin - Booking_date

In [120]:
def convert_to_date(dataset, column_name):
    dates=[]
    for index, row in dataset.iterrows():
        date = row[column_name]
        date_object = datetime.strptime(date, '%m/%d/%Y').date()
        dates.append(date_object)
    return dates
    #for i in range(0,len(data[column_name])):
     #   date_str = data[column_name].iloc[i]
      #  date_object = datetime.strptime(date_str, '%m/%d/%Y').date()
       # dates.append(date_object)
    #return dates

def calculate_date_difference(last_date, first_date):
    differences =[]
    for i in range(0,len(last_date)):
        difference = (last_date[i] - first_date[i]).days
        differences.append(difference)
    return(differences)



In [125]:
# 1. Expected_stay_days
tdata['Expected_stay_days'] = calculate_date_difference(convert_to_date(tdata,'Expected_checkout'),
                                    convert_to_date(tdata,'Expected_checkin'))

vdata['Expected_stay_days']  = calculate_date_difference(convert_to_date(vdata,'Expected_checkout'),
                                    convert_to_date(vdata,'Expected_checkin'))

testdata['Expected_stay_days']  = calculate_date_difference(convert_to_date(testdata,'Expected_checkout'),
                                    convert_to_date(testdata,'Expected_checkin'))

tdata['Reservation_gap'] = calculate_date_difference(convert_to_date(tdata,'Expected_checkin'),
                                                    convert_to_date(tdata, 'Booking_date'))

# 2. Reservation_gap_days
vdata['Reservation_gap'] = calculate_date_difference(convert_to_date(vdata,'Expected_checkin'),
                                                    convert_to_date(vdata, 'Booking_date'))

testdata['Reservation_gap'] = calculate_date_difference(convert_to_date(testdata,'Expected_checkin'),
                                                    convert_to_date(testdata, 'Booking_date'))

In [129]:
column_drop = ['Expected_checkout', 'Expected_checkin','Booking_date']
tdata.drop(column_drop, axis=1, inplace=True)
vdata.drop(column_drop, axis=1, inplace=True)
testdata.drop(column_drop, axis=1, inplace=True)

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Adults,Children,...,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate,Expected_stay_days,Reservation_gap
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,2,2,...,No,No Deposit,Online,Yes,Check-In,Yes,10,218,1,41
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,3,3,...,No,Refundable,Online,Yes,Check-In,No,0,185,1,36
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,3,3,...,No,No Deposit,Online,Yes,Check-In,No,0,119,4,3
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,4,3,...,No,Refundable,Agent,Yes,Check-In,Yes,5,144,1,12
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,1,1,...,No,No Deposit,Direct,No,Check-In,Yes,10,242,1,13


### Write to CSV

In [133]:
tdata.to_csv('train-dataset.csv',',')
vdata.to_csv('validation-dataset.csv',',')
testdata.to_csv('test-dataset.csv',',')